In [62]:
import numpy as np
import open3d as o3d
import copy
# Currently assume a feasible contact state must have at least a region for each finger
# Currently region can be defined by number
# Finger order: 0:thumb 1:index 2:middle 3:ring hence region [2,6,3,7]
# Need a mechanism that parse the region id to region a function input is action + region id output is finger tip pose. each object should have a unique action parser

class ContactStateGraph:
    def __init__(self, states):
        """
        states: np.array of region id
        """
        self.states = states
        self.state_ids = list(range(len(self.states)))
        self.graph_adj_list = self.build_graph(self.states, self.state_ids)

    # Represent the graph as adjacency list, each entry should have number indicating edge type
    def build_graph(self, states, state_ids):
        # initialize adjacency list
        graph_adj_list = {}
            
        # Build the list
        for i in range(len(self.state_ids)):
            i_id = state_ids[i]
            graph_adj_list[i_id] = []
            for j in range(len(state_ids)):
                j_id = state_ids[j]
                common_regions = (states[i_id] == states[j_id])
                num_common_regions = common_regions.sum()
                if num_common_regions != 0:
                    graph_adj_list[i_id].append((j_id, common_regions, num_common_regions))
        return graph_adj_list

    def getNeighbors(self,state_id):
        return self.graph_adj_list[state_id]

    def getState(self, state_id):
        return self.states[state_id]

    def getPathFromState(self,state_id, steps):
        self.paths = []
        self.weights = []
        self._getPathFromState(state_id, steps)
        return self.paths, self.weights
    # return all paths as well as their weights (absolute) maybe recursive?
    def _getPathFromState(self, state_id, steps, total_weight=1, current_path=[]):
        neighbors = self.getNeighbors(state_id)
        weights = np.array([n[2] for n in neighbors])
        weights = weights/weights.sum()
        current_path.append(state_id)
        for i,neighbor in enumerate(neighbors):
            if steps != 1:
                self._getPathFromState(neighbor[0],steps-1, total_weight*weights[i], current_path=copy.deepcopy(current_path))
            else:
                self.paths.append(current_path+[neighbor[0]])
                self.weights.append(total_weight*weights[i]) # Final weights


In [64]:
# Assume we have 8 region
states = []
states.append(np.array([5, 27, 26, 25]))
states.append(np.array([5, 23, 26, 29]))
states.append(np.array([5, 25, 26, 27]))
states.append(np.array([5, 29, 26, 23]))
states.append(np.array([26, 4, 5, 6]))
states.append(np.array([26, 8, 5, 2]))
states.append(np.array([26, 6, 5, 4]))
states.append(np.array([26, 2, 5, 8]))
np.save("../data/contact_states/laptop_env/dummy_states.npy", states)

#csg = ContactStateGraph(states)
#paths, weights = csg.getPathFromState(0,3)
#print(paths)
#print(weights)

In [13]:
s = 0.2 - 0.4/3
regions = np.array([[-0.2, -s, -0.2, -s, 0.05, 0.05],
                    [-0.2, -s, -s, s, 0.05, 0.05],
                    [-0.2, -s, s, 0.2, 0.05, 0.05],
                    [-s, s, -0.2, -s, 0.05, 0.05],
                    [-s, s, -s, s, 0.05, 0.05],
                    [-s, s, s, 0.2, 0.05, 0.05],
                                 [s, 0.2, -0.2, -s, 0.05, 0.05],
                                 [s, 0.2, -s, s, 0.05, 0.05],
                                 [s, 0.2, s, 0.2, 0.05, 0.05],
                                 [-0.2, -0.2, -0.2, -s, -0.05, 0.05],
                                 [-0.2, -0.2, -s, s, -0.05, 0.05],
                                 [-0.2, -0.2, s, 0.2, -0.05, 0.05],
                                 [-0.2, -s, 0.2, 0.2, -0.05, 0.05],
                                 [-s, s, 0.2, 0.2, -0.05, 0.05],
                                 [s, 0.2, 0.2, 0.2, -0.05, 0.05],
                                 [0.2, 0.2, s, 0.2, -0.05, 0.05],
                                 [0.2, 0.2, -s, s, -0.05, 0.05],
                                 [0.2, 0.2, -0.2, -s, -0.05, 0.05],
                                 [s, 0.2, -0.2, -0.2, -0.05, 0.05],
                                 [-s, s, -0.2, -0.2, -0.05, 0.05],
                                 [-0.2, -s, -0.2, -0.2, -0.05, 0.05],
                                 [-0.2, -s, -0.2, -s, -0.05, -0.05],
                                 [-0.2, -s, -s, s, -0.05, -0.05],
                                 [-0.2, -s, s, 0.2, -0.05, -0.05],
                                 [-s, s, -0.2, -s, -0.05, -0.05],
                                 [-s, s, -s, s, -0.05, -0.05],
                                 [-s, s, s, 0.2, -0.05, -0.05],
                                 [s, 0.2, -0.2, -s, -0.05, -0.05],
                                 [s, 0.2, -s, s, -0.05, -0.05],
                                 [s, 0.2, s, 0.2, -0.05, -0.05]])
fixed_axes = np.array([2,2,2,2,2,2,2,2,2,0,0,0,1,1,1,0,0,0,1,1,1,2,2,2,2,2,2,2,2,2])
np.savez("../data/regions/small_block_dummy_region.npz", regions=regions, fixed_axes=fixed_axes)

In [30]:
class SmallBlockRegionDummy:
    def __init__(self, contact_state_graph):
        self.x_range = [-0.2, 0.2]
        self.y_range = [-0.2, 0.2]
        self.z_range = [-0.05, 0.05]
        self.csg = contact_state_graph
        region_data = np.load("../data/regions/small_block_dummy_region.npz")
        self.regions = region_data["regions"]
        self.fixed_axis = region_data["fixed_axes"]

    def parse_action(self, state_id, action):
        """
        state_id: int id of contact state
        action: np.ndarray[4,2] each element is bounded within [-1,1]
        """
        state = self.csg.getState(state_id)-1 # Region ID
        print(state)
        finger_regions = self.regions[state]
        fixed_axes = self.fixed_axis[state]
        print(finger_regions)
        print(fixed_axes)
        scaled_action = (action+1) * 0.5 # Mapped to [0, 1]
        finger_tip_pos = []

        for i,region in enumerate(finger_regions):
            sub_a = scaled_action[i]
            fixed_axis = fixed_axes[i]
            if fixed_axis == 0:
                x = region[0]
                y_range = region[3] - region[2]
                y_start = region[2]
                y = y_range * sub_a[0] + y_start
                z_range = region[5] - region[4]
                z_start = region[4]
                z = z_range * sub_a[1] + z_start
            elif fixed_axis == 1:
                x_range = region[1] - region[0]
                x_start = region[0]
                x = x_range * sub_a[0] + x_start
                y = region[2]
                z_range = region[5] - region[4]
                z_start = region[4]
                z = x_range * sub_a[1] + z_start
            else:
                x_range = region[1] - region[0]
                x_start = region[0]
                x = x_range * sub_a[0] + x_start
                y_range = region[3] - region[2]
                y_start = region[2]
                y = y_range * sub_a[0] + y_start
                z = region[4]
            finger_tip_pos.append(np.array([x,y,z]))
        return finger_tip_pos

In [31]:
dummy_region = SmallBlockRegionDummy(csg)

In [32]:
dummy_region.parse_action(0, np.zeros((4,2)))

[ 4 26 25 24]
[[-0.06666667  0.06666667 -0.06666667  0.06666667  0.05        0.05      ]
 [-0.06666667  0.06666667  0.06666667  0.2        -0.05       -0.05      ]
 [-0.06666667  0.06666667 -0.06666667  0.06666667 -0.05       -0.05      ]
 [-0.06666667  0.06666667 -0.2        -0.06666667 -0.05       -0.05      ]]
[2 2 2 2]


[array([0.  , 0.  , 0.05]),
 array([ 0.        ,  0.13333333, -0.05      ]),
 array([ 0.  ,  0.  , -0.05]),
 array([ 0.        , -0.13333333, -0.05      ])]

In [ ]:
# From a starting node, we need to sample multiple path instances, and allocate number of particles go to these path.
# Need to get all paths starting from a given state and rollout k steps. As well as weight of these paths
def get_all_path_from_state(state_id, steps, ):